In [4]:
import numpy as np
import textwrap
import matplotlib.pyplot as plt

import train_integers
from activations import *

examples = 569
neurons = 30
bitwidth = 20
fraction = 16
verbosity = "low"



def twos_comp(val, bits):
    """compute the 2's complement of int value val"""
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val                         # return positive value as is

In [5]:

with open("breast_input_30neuron_20bit_16frac.mem", 'r') as input_file:
    inputs = np.zeros((examples, neurons))
    text = input_file.readlines()
    for row, line in enumerate(text):
        elements = textwrap.wrap(line, bitwidth)
        elements = np.array([int(el, 2) for el in elements])
        inputs[row] = elements
        
    inputs = inputs.astype(int)
        
with open("breast_output_30neuron_20bit_16frac.mem", 'r') as output_file:
    outputs = np.zeros((examples, neurons))
    text = output_file.readlines()
    for row, line in enumerate(text):
        elements = textwrap.wrap(line, bitwidth)
        elements = np.array([int(el, 2) for el in elements])
        outputs[row] = elements
        
    outputs = outputs.astype(int)
        
with open("weights_30x30_20bit_16frac.mem", 'r') as weight_file:
    weights = np.zeros(neurons * neurons)
    text = weight_file.readlines()
    elements = textwrap.wrap(text[0], bitwidth)
    elements = np.array([int(el, 2) for el in elements])
    weights = elements
    weights = np.array([twos_comp(w, bitwidth) for w in weights])
    weights = weights.astype(int)

# This cell prints the values which should be seen in hardware

In [6]:
iter = 10
x = inputs
y = outputs
w = weights.reshape(30, 30)
func = linear
func_der = linear_derivative
lr = 9
verbosity = "high"

for i in range(iter): 
    sample = i % len(x)

    z0     = x[sample]
    target = y[sample]

    z1 = np.matmul(w, z0)
    a1 = func(z1)
    z1 = np.right_shift(z1, fraction)
    a1 = np.right_shift(a1, fraction)

    delta = (target - a1) * func_der(z1)
    delta = np.round(delta).astype(int)

    updates = delta.reshape(len(delta), 1) * z0.reshape(1, len(z0))
    updates = np.right_shift(updates, fraction)
    updates_shifted = np.right_shift(updates, lr)
    
    w_old = w
    w = w + updates_shifted

    if verbosity == "low": 
        #print np.sum(np.abs(target - a1)) / 2.0**fraction
        print np.argmax(target[:2]) == np.argmax(a1[:2])
    if verbosity == "high":
        print("target", target)
        print("z0", z0)
        print("z1", z1)
        print('errors', target - z1)
        print("delta", delta)
        print("updates", updates)
        print("shifted", updates_shifted)
        print("w_old", w_old)
        print("w", w)


('target', array([65536,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0]))
('z0', array([43877, 36655, 43006, 29086, 40829, 26354, 24476, 28481, 41886,
       41242, 18684, 25905, 13396, 12558, 16997, 19786,  8805, 22768,
       19781,  9916, 41205, 40917, 37907, 24695, 40923, 21450, 20477,
       38465, 29687, 26258]))
('z1', array([ 10274,  14897,   6408,   5809,   -655, -12866,  -2923,   2202,
        -7603,  -4851,  -4082,   3102,    853,  -3441,   2342, -11916,
         3219,  -3719,   6560,  -4978,   -398,   -729,   6676,  -5793,
        13697,  -4196,   2660,     80,      1,  13112]))
('errors', array([ 55262, -14897,  -6408,  -5809,    655,  12866,   2923,  -2202,
         7603,   4851,   4082,  -3102,   -853,   3441,  -2342,  11916,
        -3219,   3719,  -6560,   4978,    398,    729,  -6676,   5793,
 